<a href="https://colab.research.google.com/github/tchowdhury/AWS-Projects/blob/master/CIFAR10_Connected_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import necessary pytorch libraries
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms

In [3]:
# Transform the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.48216, 0.44653), ((0.24703, 0.24349, 0.26159)))])

In [4]:
# Prepare training set and testing set
trainset = torchvision.datasets.CIFAR10('cifar', train=True, 
                                      download=True, transform=transform)
testset = torchvision.datasets.CIFAR10('cifar',  train=False,
                                     download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting cifar/cifar-10-python.tar.gz to cifar
Files already downloaded and verified


In [6]:
# Prepare training loader and testing loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [7]:
# Compute the shape of the training set and testing set
trainset_shape = trainloader.dataset.data.shape
testset_shape = testloader.dataset.data.shape

In [8]:
# Print the computed shapes
print(trainset_shape, testset_shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [9]:
# import necessary pytorch neural network libraries
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
# Define the class Net
# The dataset contains images of shape (32, 32, 3), so you should deduct the size of the input layer. For hidden layer use 500 units, while for output layer use 10 units (1 for each class)
class Net(nn.Module):
    def __init__(self):    
    	# Define all the parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 500)
        self.fc2 = nn.Linear(500,10)

    def forward(self, x):   
    	# Do the forward pass
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model = Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

In [12]:
for epoch in range(15): # loop over the dataset multiple times
    for batch_idx, data_target in enumerate(trainloader):
        data = data_target[0]
        target = data_target[1]
        data = data.view(-1, 32 * 32 * 3)
        optimizer.zero_grad()

        # Complete a forward pass
        output = model(data)

        # Compute the loss, gradients and change the weights
        loss = criterion(output,target)
        # print("for Epoch %e the Loss is %l" % (epoch, loss))
        loss.backward()
        optimizer.step()

In [13]:
# Set the model in eval mode
correct, total = 0, 0
predictions = []
model.eval()

Net(
  (fc1): Linear(in_features=3072, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [14]:
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    
    # Put each image into a vector
    inputs = inputs.view(-1, 32 * 32 * 3)
    
    # Do the forward pass and get the predictions
    outputs = model(inputs)
    _, outputs = torch.max(outputs.data, 1)
    predictions.append(outputs)
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 52 %
